In [1]:
# Load contractions model
from pycontractions import Contractions

cont = Contractions(api_key="glove-twitter-100")
cont.load_models()

### Parameters

In [2]:
file = 'Data/c4.csv'
ngram = (1,3)
mindf = 2
maxdf = 0.8

### Load

In [3]:
import pandas as pd

df = pd.read_csv(file,sep=",")
display(df.head())

,id,text,author,date,keywords,organization,rating
0,http://data.gesis.org/claimskg/creative_work/9...,California’s Prop 55 'prevents $4 billion in n...,Yes on 55,03/26/2055,"education,state finances",politifact,MIXTURE
1,http://data.gesis.org/claimskg/creative_work/e...,Says an election clerk in Green Bay refused to...,Our Democracy 2020,03/26/2020,"bipartisanship,city government,education,elect...",politifact,MIXTURE
2,http://data.gesis.org/claimskg/creative_work/0...,Says state schools superintendent candidate Do...,One Wisconsin Now,12/03/2019,"corporations,education,small business,state bu...",politifact,MIXTURE
3,http://data.gesis.org/claimskg/creative_work/f...,"'The Walton family, which owns Wal-Mart, contr...",One Wisconsin Now,11/27/2019,"corporations,economy,income,labor,wealth",politifact,TRUE
4,http://data.gesis.org/claimskg/creative_work/c...,In his first meeting with University of Wiscon...,One Wisconsin Now,11/09/2019,"education,state budget",politifact,FALSE


In [4]:
df5 = df.copy().sort_values(by="author")
df6 = df5.loc[(df5['organization'] != 'snopes')]
authors = df6["author"].unique()
#display(df6)

print(df6.shape)
len(authors)

(6258, 7)


2076

In [5]:
df1 = df.copy()
df1 = df1.loc[df1['rating'] != 'OTHER']
display(df1.head())

,id,text,author,date,keywords,organization,rating
0,http://data.gesis.org/claimskg/creative_work/9...,California’s Prop 55 'prevents $4 billion in n...,Yes on 55,03/26/2055,"education,state finances",politifact,MIXTURE
1,http://data.gesis.org/claimskg/creative_work/e...,Says an election clerk in Green Bay refused to...,Our Democracy 2020,03/26/2020,"bipartisanship,city government,education,elect...",politifact,MIXTURE
2,http://data.gesis.org/claimskg/creative_work/0...,Says state schools superintendent candidate Do...,One Wisconsin Now,12/03/2019,"corporations,education,small business,state bu...",politifact,MIXTURE
3,http://data.gesis.org/claimskg/creative_work/f...,"'The Walton family, which owns Wal-Mart, contr...",One Wisconsin Now,11/27/2019,"corporations,economy,income,labor,wealth",politifact,TRUE
4,http://data.gesis.org/claimskg/creative_work/c...,In his first meeting with University of Wiscon...,One Wisconsin Now,11/09/2019,"education,state budget",politifact,FALSE


In [6]:
t = df1.loc[(df1['rating'] == 'TRUE')]['id'].count()
f = df1.loc[(df1['rating'] == 'FALSE')]['id'].count()
m = df1.loc[(df1['rating'] == 'MIXTURE')]['id'].count()

print('True claims : %d' % t)
print('False claims : %d' % f)
print('Mixture claims : %d' % m)
print('TF claims : %d' % (t+f))

True claims : 979
False claims : 1613
Mixture claims : 3587
TF claims : 2592


### Preprocess

In [7]:
import re

# def preprocess(sentence):
#     sentence = sentence.lower().strip()
#     translator = str.maketrans('’', "'", '')
#     sentence = sentence.translate(translator)
#     sentence = re.sub("u\.s\.","united states",sentence)
#     sentence = list(cont.expand_texts([sentence],precise=True))[0]
#     sentence = re.sub("[^a-zA-Z0-9_.’,]|(?<!\d)\.(?!\d)|(?<!\w)-(?!\w)|(?<!\d)\,(?!\d)",' ',sentence)
#     sentence = re.sub(",",'',sentence)
#     sentence = re.sub('\s+', ' ', sentence).strip()
#     sentence = re.sub(" s ",' ',sentence)
#     if sentence[0:5] == 'says ':
#         sentence = sentence[5:]
#     return sentence

def preprocess(sentence):
    sentence = sentence.lower().strip()
    translator = str.maketrans('’', "'", '')
    sentence = sentence.translate(translator)
    sentence = re.sub("u\.s\.","united states",sentence)
    sentence = list(cont.expand_texts([sentence],precise=True))[0]
    sentence = re.sub("[^a-zA-Z0-9_.’,]|(?<!\d)\.(?!\d)|(?<!\w)-(?!\w)|(?<!\d)\,(?!\d)",' ',sentence)
    sentence = re.sub(",",'',sentence)
    sentence = re.sub("\.",'',sentence)
    sentence = re.sub(" a ",' ',sentence)
    sentence = re.sub('\s+', ' ', sentence).strip()
    sentence = re.sub(" s ",' ',sentence)
    if sentence[0:5] == 'says ':
        sentence = sentence[5:]
    sentence = ' '.join([w for w in sentence.split() if len(w)>1])
    return sentence

In [8]:
for index in df1.index:
    df1.at[index,'text'] = preprocess(df1.at[index,'text'])
    rating = df1.at[index,'rating']
    if rating == 'FALSE':
        df1.at[index,'rating'] = 0
    if rating == 'TRUE':
        df1.at[index,'rating'] = 1
    if rating == 'MIXTURE':
        df1.at[index,'rating'] = 2

In [9]:
for index in df1.index:
    if index < 20:
        print("%s\n" % df1.at[index,'text'])

california prop 55 prevents billion in new cuts to our schools

an election clerk in green bay refused to allow early voting on the uw green bay campus because she was afraid it would help democrats showing outrageous partisan bias

state schools superintendent candidate don pridemore voted for the largest cuts to public education in our state history and for 23 billion in tax breaks for corporations and the wealthy

the walton family which owns wal mart controls fortune equal to the wealth of the bottom 42 percent of americans combined

in his first meeting with university of wisconsin system officials republican governor elect scott walker told them to prepare for cuts

scott walker was elected governor on promise not to take campaign contributions from the date of his inauguration until the signing of the state budget but broke it by taking nearly 56 million

obama adviser david plouffe took large piles of cash from joint venture partner of company owned by iran revolutionary guard


### Create additional dataframes

In [10]:
#df1 = true/false/mixed
#df2 = true/false
#df3 = true-false/mixed

In [11]:
df2 = df1.copy()
df2 = df2.loc[(df2['rating'] == 0) | (df2['rating'] == 1)]
df2.head()

,id,text,author,date,keywords,organization,rating
3,http://data.gesis.org/claimskg/creative_work/f...,the walton family which owns wal mart controls...,One Wisconsin Now,11/27/2019,"corporations,economy,income,labor,wealth",politifact,1
4,http://data.gesis.org/claimskg/creative_work/c...,in his first meeting with university of wiscon...,One Wisconsin Now,11/09/2019,"education,state budget",politifact,0
5,http://data.gesis.org/claimskg/creative_work/a...,scott walker was elected governor on promise n...,One Wisconsin Now,09/26/2019,"campaign finance,elections,state budget",politifact,0
9,http://data.gesis.org/claimskg/creative_work/f...,deer baiting planned for block island by dem i...,Richard August,08/26/2019,"animals,crime,environment,government regulatio...",politifact,1
14,http://data.gesis.org/claimskg/creative_work/8...,oregon is one of only three states that allow ...,Stop Gillnetting Now,05/30/2019,"environment,jobs,message machine 2012,recreation",politifact,0


In [12]:
df3 = df1.copy()
for index in df3.index:
    rating = df3.at[index,'rating']
    if rating == 1:
        df3.at[index,'rating'] = 0
    if rating == 2:
        df3.at[index,'rating'] = 1
        
df3.head()

,id,text,author,date,keywords,organization,rating
0,http://data.gesis.org/claimskg/creative_work/9...,california prop 55 prevents billion in new cut...,Yes on 55,03/26/2055,"education,state finances",politifact,1
1,http://data.gesis.org/claimskg/creative_work/e...,an election clerk in green bay refused to allo...,Our Democracy 2020,03/26/2020,"bipartisanship,city government,education,elect...",politifact,1
2,http://data.gesis.org/claimskg/creative_work/0...,state schools superintendent candidate don pri...,One Wisconsin Now,12/03/2019,"corporations,education,small business,state bu...",politifact,1
3,http://data.gesis.org/claimskg/creative_work/f...,the walton family which owns wal mart controls...,One Wisconsin Now,11/27/2019,"corporations,economy,income,labor,wealth",politifact,0
4,http://data.gesis.org/claimskg/creative_work/c...,in his first meeting with university of wiscon...,One Wisconsin Now,11/09/2019,"education,state budget",politifact,0


In [13]:
dfs = dict()
dfs['3 Classes'] = df1
dfs['2 Classes'] = df2
dfs['2 Mixed Classes'] = df3

### Vectors

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorize(df):
    vectorizer = TfidfVectorizer(ngram_range=ngram,min_df=mindf,max_df=maxdf)
    vectors = vectorizer.fit_transform(df['text']).toarray()
    #print(vectorizer.get_feature_names())
    return vectors

### Classification

In [15]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

def classify(df, key):
    X = vectorize(df)
    y = df["rating"]
    y = y.astype('int')
    clf = LinearSVC(random_state=0, tol=1e-5)
    k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=None)
    score = cross_val_score(clf, X, y, cv=k_fold, scoring='accuracy')
    print('%s - Mean accuracy: %f, Deviation: %f' % (key,score.mean(),score.std()))

In [16]:
for key in dfs:
    classify(dfs[key], key)

3 Classes - Mean accuracy: 0.551233, Deviation: 0.013827
2 Classes - Mean accuracy: 0.650431, Deviation: 0.029969
2 Mixed Classes - Mean accuracy: 0.562215, Deviation: 0.026105


### Feature Selection

In [17]:
from sklearn.feature_selection import mutual_info_classif

def get_top_features(df):
    vectorizer = TfidfVectorizer(ngram_range=ngram,min_df=mindf,max_df=maxdf)
    vectors = vectorizer.fit_transform(df['text']).toarray()
    feature_scores = mutual_info_classif(vectors, df['rating'].astype('int'))
    
    for score, fname in sorted(zip(feature_scores, vectorizer.get_feature_names()), reverse=True)[:30]:
        print(fname, score)

In [18]:
# for key in dfs:
#     print("********** %s **********" % key)
#     get_top_features(dfs[key])
#     print()